## DB 1:N 실습문제

기존 movies_blog 프로젝트를 이어서 진행합니다.

Movie Review에 댓글을 달 수 있는 기능을 추가하려고 한다. 

아래 문제를 풀면서 기능을 구현하세요. 

#### 0. 로그인/비로그인 구별하기
0. base.html 의 모든 a 태그가 url DTL을 활용해서 동작하도록 수정하세요.

1. top bar 에서 로그인한 경우에는 [LOGOUT]이 보이고, 비로그인 상태에서는 [LOGIN]이 보이도록 한다.

2. top bar 에서 로그인한 경우에는 [PROFILE] 이 보이고, 클릭할 경우 accounts/index로 이동하여 유저정보 페이지로 이동한다. ( 비로그인상태로 accounts/index로 이동할 경우에는 로그인 페이지로 리다이렉션하도록 한다.)


- 로그인한 경우  

    <img src="practice_images/image63.jpg" width="30%" height="1%"/>

- 비로그인인 경우  

    <img src="practice_images/image64.jpg" width="30%" height="1%"/>

3. 비로그인상태에서는 리뷰를 작성할 수 없도록 한다. 

- 로그인한 경우  

    <img src="practice_images/image65.jpg" width="30%" height="1%"/>

- 비로그인인 경우  

    <img src="practice_images/image66.jpg" width="30%" height="1%"/>

0. 모두 url DTL을 사용하기 위해서 templates/base.html을 아래와 같이 바꾼다.
( templates/base.html)

    ```html
    <!DOCTYPE html>
    <html lang="en">
    <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>영화 블로그 프로젝트</title>
    {% load static %}
    <link rel="stylesheet" href="{% static 'css/styles.css' %}">
    </head>
    <body>
    <a href="{% url "movies:index" %}">[MAIN]</a> | 
    <a href="{% url "movies:create_movie" %}">[CRAETE]</a>
    <a href="{% url "reviews:index" %}">[REVIEWS]</a>
    <a href="{% url "accounts:login" %}">[LOGIN]</a>
    {% block content %}
    {% endblock content %}
    </body>
    </html>
    ```

1. 로그인/비로그인에 따라 topbar가 다르게 보여야함으로 templates/base.html을 수정한다.
( teampltes/base.html )

    ```html
        {% if user.is_authenticated %}
        <a href="{% url "accounts:logout" %}">[LOGOUT]</a>
        {% else %}
        <a href="{% url "accounts:login" %}">[LOGIN]</a>
        {% endif %}
    ```

2. top bar에 [PROFILE]이 보이도록 하게 하기 위해서 templates/base.html을 아래와 같이 수정한다.
( templates/base.html)

    ```html
        {% if user.is_authenticated %}
        <a href="{% url "accounts:index" %}">[PROFILE]</a>
        <a href="{% url "accounts:logout" %}">[LOGOUT]</a>
        {% else %}
        <a href="{% url "accounts:login" %}">[LOGIN]</a>
        {% endif %}
    ```

3. 로그인한 경우에만 accounts/index/ 경로에 접근할 수 있도록 accounts/views.py를 수정한다. 
( accounts/views.py )

    ```python
    # accounts/views.py
    from django.contrib.auth.decorators import login_required
    @login_required
    def index(request):
        return render(request, "accounts/index.html")
    ```

4. 비로그인상태에서는 리뷰를 작성할 수 없도록 하기 위해서 reviews/index.html을 수정한다.
( reviews/index.html )

    ```html
        {% if user.is_authenticated %}
            <a href="{% url 'reviews:create' %}">리뷰 작성하기</a>
        {% endif %}
    ```

5. 리뷰 작성 함수에서도 막아야함으로 reviews/views.py의 create 함수도 수정한다. 
( reviews/views.py 의 create 함수 )

    ```python
    from django.contrib.auth.decorators import login_required
    @login_required
    def create(request):
        if request.method == "POST":
            form = ReviewForm(request.POST)
            if form.is_valid():
                review = form.save()
                return redirect("reviews:index")
        else:
            form = ReviewForm()
        context = {
            "form": form
        }
        return render(request, 'reviews/create.html', context)
    ```

#### 1. Review 와 Comment 관계 형성 ( 1: N )
1. 리뷰와 댓글을 1:N 관계를 형성한다. 
2. 댓글은 content와 작성날짜(데이터 생성 시 자동생성)으로 구성된다.
3. 리뷰 상세페이지에서 댓글을 작성 및 댓글 목록을 볼 수 있도록 한다. 

    - 댓글 작성 및 목록 이미지  

        <img src="practice_images/image67.jpg" width="30%" height="1%"/>

4. 댓글을 삭제할 수 있어야 한다. 

    - 댓글 삭제 태그 이미지  

        <img src="practice_images/image67.jpg" width="30%" height="1%"/>
    
5. 리뷰 목록 페이지에서 각 리뷰 제목 옆에 댓글 개수를 괄호로 표시한다. 

    - 리뷰 제목 옆 댓글 개수 괄호 이미지   

        <img src="practice_images/image68.jpg" width="30%" height="1%"/>

6. 로그인한 유저만 댓글을 작성할 수 있도록 한다. 

    - 비로그인 유저는 댓글 작성할 수 없는 이미지 

        <img src="practice_images/image69.jpg" width="30%" height="1%"/>


1. Comment 모델 생성 및 관계 설정
( reviews/models.py 에 아래 코드를 추가한다. )

    ```python
    class Comment(models.Model):
        content = models.TextField()
        created_at = models.DateTimeField(auto_now_add=True)
        review = models.ForeignKey(Review, on_delete=models.CASCADE, related_name="comments")
    ```
2. migrations 및 migrate로 DB에 올바르게 형성됐는지 확인한다.
3. Comment 모델폼을 생성한다. 
( reviews/forms.py 에 아래 코드를 추가한다. )

    ```python
    class CommentForm(forms.ModelForm):
        class Meta:
            model = Comment
            fields = ("content", )
            widgets = {
                'content': forms.TextInput(attrs={"placeholder" : "댓글을 입력하세요"}),
            }
    ```

4. CommentForm을 detail.html에 전달해줘야함으로 detail 함수를 수정한다.
( reviews/views.py 의 detail 함수)

    ```python
    from .forms import CommentForm
    def detail(request, review_idx):
        review = Review.objects.get(pk=review_idx)
        form = CommentForm()
        context = {
            "review": review,
            "form": form
        }
        return render(request, "reviews/detail.html", context)
    ```

5. detail.html 에 댓글 작성 구현을 위해 form을 활용하여 아래 코드를 추가한다. 
( reviews/detail.html )

    ```html
    <form action="{% url "reviews:create_comment" review.pk %}" method="POST">
        {% csrf_token %}
        {{ form }}
        <input type="submit" value="댓글 작성">
    </form>
    ```

6. detail.html에서 필요한 reviews:create_comment url 을 위해서 reviews/urls.py에 url 경로를 추가한다.
( reviews/urls.py )

    ```python
        urlpatterns = [
    path('create_comment/<int:review_idx>', views.create_comment, name="create_comment"),  # 이하 생략  
    ]   
    ```

7. url에 등록한 create_comment 함수를 작성한다.
( reviews.views.py)

    ```python
    # reviews/views.py
    def create_comment(request, review_idx):
        review = Review.objects.get(pk=review_idx)
        form = CommentForm(request.POST)
        if form.is_valid():
            comment=form.save(commit=False)
            comment.review = review
            comment.save()
            return redirect('reviews:detail', review_idx)
        context = {
            'review': review,
            'form': form
        }
        return render(request, 'reviews/detail.html', context)
    ```

8. 댓글 목록을 리뷰 상세 페이지에서 보여주기 위해 리뷰의 댓글을 조회한 후에 반환하기 위해서 detail 함수를 수정한다.
( reviews/views.py 의 detail 함수 )

    ```python
    def detail(request, review_idx):
        review = Review.objects.get(pk=review_idx)
        comments = review.comments.all()
        form = CommentForm()
        context = {
            "review": review,
            "form": form,
            "comments": comments
        }
        return render(request, "reviews/detail.html", context)    
    ```

9. 리뷰 상세 페이지에서 context를 통해 전달된 comments를 출력한다.
( reviews/detail.html )
    
    ```html
    <h4>댓글 목록</h4>
    <ol>
        {% for comment in comments %}
            <li>댓글: {{ comment.content }}</li>
        {% endfor %}
    </ol>
    ```

10. 본인이 작성한 댓글은 삭제 버튼이 보여지기 위해 detail.html을 아래와 같이 수정한다.
( reviews/detail.html )

    ```html
    <h4>댓글 목록</h4>
    <ol>
        {% for comment in comments %}
            <li>댓글: {{ comment.content }}</li>
            <form action="{% url "reviews:delete_comment" review_idx=review.pk comment_idx=comment.pk %}" mehtod="POST">
                {% csrf_token %}
                <input type="submit" value="댓글 삭제">
            </form>
        {% endfor %}
    </ol>
    ```

11. delete_comment url를 추가한다.
( reviews/urls.py )

    ```python
    urlpatterns = [
    path('detail/<int:review_idx>/delete_comment/<int:comment_idx>', views.delete_comment, name="delete_comment"),] # 이하생략
    ```

12. delete_comment url에서 필요로 하는 delete_comment 함수를 생성한다.
( reviews/views.py 에 delete_comment 함수 생성)

    ```python
    # reviews/views.py
    from .models import Comment
    def delete_comment(request, review_idx, comment_idx):
        comment = Comment.objects.get(pk=comment_idx)
        comment.delete()
        return redirect('reviews:detail', review_idx)
    ```

13. 리뷰 목록 페이지에서 댓글 개수를 출력하기 위해서 index.html을 수정한다.
( reviews/index.html )

    ```html
    {% extends "base.html" %}

    {% block content %}
        <br>
        <a href="{% url 'reviews:create' %}">리뷰 작성하기</a>
        <h2>리뷰 목록</h2>
        <ul>
            {% for review in reviews %}
            <li>
                <p>작성일 : 작성 시간 출력하기</p>
                <p>영화 제목 : {{ review.movie_title }}</p>
                <a href="{% url "reviews:detail" review_idx=review.pk %}">리뷰 제목  : {{ review.review_title }}</a>
                ( {{ review.comments.all|length }} )
                <p>리뷰 내용: {{ review.review_content }}</p>
            </li>
            <hr>
            {% empty %}
            <li>작성된 리뷰가 없습니다.</li>
            {% endfor %}
        </ul>
    {% endblock content %}
    ```

14. 로그인한 유저만 댓글을 달 수 있게 하기 위해서 reviews/detail.html을 수정한다. 
( reviews/detail.html )

    ```html
        <a href="{% url 'reviews:index' %}">[리뷰 목록 보러가기]</a>  
        <hr>
        {% if request.user.is_authenticated %}
            <form action="{% url "reviews:create_comment" review.pk %}" method="POST">
                {% csrf_token %}
                {{ form }}
                <input type="submit" value="댓글 작성">
            </form>
            <hr>
        {% endif %}
        <h4>댓글 목록</h4>
    ```


#### 2. Review 와 User 관계 형성 ( 1: N )
1. 리뷰와 유저를 1:N 관계를 형성한다. 
2. 리뷰를 작성한 후 리뷰 목록 및 상세 페이지에서 작성자 아이디가 보이도록한다.
 
    - 리뷰 목록 및 상세 페이지에 작성자 아이디가 추가된 이미지  

        <img src="practice_images/image69.jpg" width="30%" height="1%"/>
        <img src="practice_images/image68.jpg" width="30%" height="1%"/>
    

3. 리뷰 상세 페이지에서 본인이 작성한 리뷰에 한해서 수정/삭제 버튼이 보이고, 수정/삭제가 가능하도록 한다. 

    - 본인이 작성한 리뷰의 상세보기 페이지  

        <img src="practice_images/image70.jpg" width="30%" height="1%"/>

    - 본인이 작성하지 않은 리뷰의 상세보기 페이지  
    
        <img src="practice_images/image71.jpg" width="30%" height="1%"/>

    

1. Review Model을 아래와 같이 수정한다.
( reviews/models.py )

    ```python
    from django.conf import settings

    class Review(models.Model):
        movie_title = models.CharField(max_length=20)
        review_title = models.TextField()
        review_content = models.TextField()
        created_at = models.DateTimeField(auto_now_add=True)
        updated_at = models.DateTimeField(auto_now=True)
        user = models.ForeignKey(settings.AUTH_USER_MODEL, on_delete=models.CASCADE, related_name="reviews")
    ```

2. makemigrations/migrate를 진행한다. (기본값은 관리자 계정의 id 인 1로 설정한다.)
3. 리뷰 목록 페이지에서 작성자를 출력하기 위해서 index.html를 아래와 같이 수정한다.
( reviews/index.html )

    ```html
    {% extends "base.html" %}

    {% block content %}
        <br>
        <a href="{% url 'reviews:create' %}">리뷰 작성하기</a>
        <h2>리뷰 목록</h2>
        <ul>
            {% for review in reviews %}
            <li>
                <p>작성일 : 작성 시간 출력하기</p>
                <p>작성자 : {{ review.user }}</p>
                <p>영화 제목 : {{ review.movie_title }}</p>
                <a href="{% url "reviews:detail" review_idx=review.pk %}">리뷰 제목  : {{ review.review_title }}</a>
                ( {{ review.comments.all|length }} )
                <p>리뷰 내용: {{ review.review_content }}</p>
            </li>
            <hr>
            {% empty %}
            <li>작성된 리뷰가 없습니다.</li>
            {% endfor %}
        </ul>
    {% endblock content %}
    ```

4. 영화 상세페이지에서 작성자를 출력하기 위해서 아래 코드로 수정한다.
( reviews/detail.html )

    ```html
    <p>영화제목: {{ review.movie_title }}</p>
    <p>작성자: {{ review.user }}</p>
    ```

5. 본인의 리뷰만 수정/삭제가 가능하도록 수정하기 위해서 reviews/detail.html을 수정한다.
( reviews/detail.html)

    ```html
        <p>최근 리뷰 업데이트 시간 : 업데이트 시간이 나타나도록 하세요.</p>
        <hr>
        {% if request.user == review.user %}
            <a href="{% url 'reviews:update' review_idx=review.pk %}">[리뷰 수정하기]</a>
            <br>
            <a href="{% url 'reviews:delete' review_idx=review.pk %}" class="review-delete-btn">[리뷰 삭제하기]</a>
            <br>
        {% endif %}
        <a href="{% url 'reviews:index' %}">[리뷰 목록 보러가기]</a>  
        <hr>
    ```

6. 리뷰 생성 시 유저 정보를 포함시키기 위해서 reviews/views.py의 create함수를 수정한다.
(reviews/views.py 의 create 함수 )

    ```python
    from django.contrib.auth.decorators import login_required
    @login_required
    def create(request):
        if request.method == "POST":
            form = ReviewForm(request.POST)
            if form.is_valid():
                review = form.save(commit=False)
                review.user = request.user
                review.save()
                return redirect("reviews:index")
        else:
            form = ReviewForm()
        context = {
            "form": form
        }
        return render(request, 'reviews/create.html', context)
    ```



#### 3. Comment 와 User 관계 형성 ( 1: N )
1. 댓글과 유저를 1:N 관계를 형성한다. 
2. 댓글 목록에서 작성자가 보이도록 한다. 
3. 댓글 목록에서 본인이 작성한 댓글은 삭제버튼이 보이며, 본인만 삭제할 수 있도록 한다. 

- 본인이 작성한 댓글과 남이 작성한 댓글을 볼 때 삭제버튼의 유뮤  

    <img src="practice_images/image60.jpg" width="30%" height="1%"/>  

1. 댓글과 유저의 관계를 형성하기 위해서 reviews/models.py 의 Comment Class를 수정한다.
( reviews/models.py )

    ```python
    class Comment(models.Model):
        content = models.TextField()
        created_at = models.DateTimeField(auto_now_add=True)
        review = models.ForeignKey(Review, on_delete=models.CASCADE, related_name="comments")
        user = models.ForeignKey(settings.AUTH_USER_MODEL, on_delete=models.CASCADE, related_name="comments")
    ```

2. makemigrations/migrate 후에 DB reviews_comment 테이블에 user_id가 추가됐는 지 확인 

3. 댓글 작성 시에 작성자 정보를 포함해야하기 때문에 reviews/views.py 의 create_comment 함수를 아래와 같이 수정한다. 
( reviews/views.py )

    ```python
    # reviews/views.py
    def create_comment(request, review_idx):
        review = Review.objects.get(pk=review_idx)
        form = CommentForm(request.POST)
        if form.is_valid():
            comment=form.save(commit=False)
            comment.review = review
            comment.user = request.user
            comment.save()
            return redirect('reviews:detail', review_idx)
        context = {
            'review': review,
            'form': form
        }
        return render(request, 'reviews/detail.html', context)    
    ```

4. 댓글 목록에서 작성자가 보여야함으로 reviews/detail.html 을 수정한다. 
( reviews/detail.html ) 

    ```html
        <h4>댓글 목록</h4>
        <ol>
            {% for comment in comments %}
                <li>{{ comment.user.username }} : {{ comment.content }}</li>
                <form action="{% url "reviews:delete_comment" review_idx=review.pk comment_idx=comment.pk %}" mehtod="POST">
                    {% csrf_token %}
                    <input type="submit" value="댓글 삭제">
                </form>
            {% endfor %}
        </ol>
    ```

5. 본인의 댓글만 삭제버튼이 보여줘야하기 때문에 reviews/detail.html을 수정한다.
( reviews/detail.html)

    ```html
        <h4>댓글 목록</h4>
        <ol>
            {% for comment in comments %}
                <li>{{ comment.user.username }} : {{ comment.content }}</li>
                {% if request.user == comment.user %}
                <form action="{% url "reviews:delete_comment" review_idx=review.pk comment_idx=comment.pk %}" mehtod="POST">
                    {% csrf_token %}
                    <input type="submit" value="댓글 삭제">
                </form>
                {% endif %}
            {% endfor %}
        </ol>
    ```

6. 본인의 댓글만 삭제 가능하도록 reviews/views.py의 delete_comment 함수도 수정하기
( reviews/views.py 의 delete_comment 함수 )

    ```python
    # reviews/views.py
    from .models import Comment
    
    def delete_comment(request, review_idx, comment_idx):
        comment = Comment.objects.get(pk=comment_idx)
        if request.user == comment.user:
            comment.delete()
        return redirect('reviews:detail', review_idx)
    ```


#### 4. 유저 프로필 페이지 수정하기
1. 본인이 작성한 리뷰 목록을 볼 수 있도록 한다.  
- 작성한 리뷰가 없으면 "작성한 리뷰가 없습니다"를 출력한다.
- 리뷰 제목을 클릭하면 리뷰 상세보기로 넘어가도록 한다.

2. 본인이 작성한 댓글 목록을 볼 수 있도록 한다.  
- 작성한 댓글이 없으면 "작성한 댓글이 없습니다"를 출력한다.
- 댓글 내용을 클릭하면 댓글이 달린 리뷰 상세보기로 넘어가도록 한다.


작성한 리뷰나 댓글이 있는 경우  
    <img src="practice_images/image61.jpg" width="30%" height="1%"/>

작성한 리뷰나 댓글이 없는 경우  
    <img src="practice_images/image62.jpg" width="30%" height="1%"/>  

1. 본인이 작성한 리뷰와 댓글 목록을 뿌려주기 위해서 reviews/views.py 에서 각 리뷰목록과 댓글 목록을 조회해서 랜더링해준다.
( accounts/views.py )

    ```python
    # accounts/views.py
    from django.contrib.auth.decorators import login_required
    @login_required
    def index(request):
        reviews = request.user.reviews.all()
        comments = request.user.comments.all()
        context = {
            'reviews': reviews,
            'comments': comments
        }
        return render(request, "accounts/index.html", context)
    ```

2. context로 넘겨받은 값들을 출력하기 위해서 accounts/index.html을 아래와같이 수정한다.

    ```html
    <!-- accounts/index.html-->
    {% extends "base.html" %}
    {% block content %}
    <h1>회원정보 페이지</h1>
    <hr>
    <h4>계정이름: {{ user.username}}</h4>
    <h4>이메일: {{ user.email}}</h4>
    <hr>
    <h2>작성한 리뷰 목록</h2>
    {% if reviews %}
    <ol>
        {% for review in reviews %}
            <li><a href="{% url "reviews:detail" review.pk %}">리뷰 제목: {{ review.review_title}}</a>
            </li>
        {% endfor %}
    </ol>
    {% else %}
        <p>작성한 리뷰가 없습니다.</p>
    {% endif %}
    <hr>
    <h2>작성한 댓글 목록</h2>
    {% if comments %}
    <ol>
        {% for comment in comments %}
            <li><a href="{% url "reviews:detail" comment.review.id %}">댓글: {{ comment.content}}</a>
            </li>
        {% endfor %}
    </ol>
    {% else %}
        <p>작성한 댓글이 없습니다.</p>
    {% endif %}
    <hr>
    <form action="{% url "accounts:update" %}" mehtod="POST">
        {% csrf_token %}
        <input type="submit" value="회원정보 수정하기">
    </form><br>
    <form action="{% url "accounts:logout" %}" mehtod="POST">
        {% csrf_token %}
        <input type="submit" value="로그아웃하기">
    </form><br>
    <form action="{% url "accounts:delete" %}" mehtod="POST">
        {% csrf_token %}
        <input type="submit" value="회원탈퇴하기">
    </form><br>
    {% endblock content %}

    ```